In [1]:
import folium

In [2]:
import ee

ee.Authenticate()

ee.Initialize()

Enter verification code:  4/1AfJohXncnic5q6XQ5R3_eqQF3FBCPqOjnQj8yOIFdGZO3G1upVe_JE-H-60



Successfully saved authorization token.


In [3]:
# import ee
# ee.Initialize()

# Define the corners of the large ROI
top_left = (70, 45)
bottom_right = (110, 23)

# Define the grid size
cols = 5  # Number of columns
rows = 4  # Number of rows

# Calculate the step size for each grid cell
width_step = (bottom_right[0] - top_left[0]) / cols
height_step = (top_left[1] - bottom_right[1]) / rows


In [7]:
def create_grid(top_left, bottom_right, width_step, height_step, cols, rows):
    grid = []
    for i in range(cols):
        for j in range(rows):
            # Calculate corners of the cell
            left = top_left[0] + i * width_step
            right = left + width_step
            top = top_left[1] - j * height_step
            bottom = top - height_step

            # Create a rectangle for this cell
            cell = ee.Geometry.Rectangle([left, bottom, right, top])
            grid.append(cell)
    return grid

grid = create_grid(top_left, bottom_right, width_step, height_step, cols, rows)
roi = ee.Geometry.Rectangle([70, 45, 110, 23])


In [8]:
modis_snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')
filtered_snow_cover = modis_snow_cover.filterDate('2012-01-01', '2013-12-31')


In [9]:
def yearly_composite(year):
    start_date = ee.Date.fromYMD(year, 1, 1)
    end_date = start_date.advance(1, 'year')
    return filtered_snow_cover.filterDate(start_date, end_date).max().clip(roi).set('year', year)

years = ee.List.sequence(2012, 2013)
annual_snow_cover = ee.ImageCollection.fromImages(years.map(yearly_composite))


In [10]:
for i, cell in enumerate(grid):
    for year in range(2012, 2013):
        image_to_export = annual_snow_cover.filter(ee.Filter.eq('year', year)).first()
        task = ee.batch.Export.image.toDrive(**{
            'image': image_to_export,
            'description': f'SnowCover_{year}_Section_{i}',
            'folder': 'EarthEngineData',
            'scale': 30,
            'region': cell,
            'fileFormat': 'GeoTIFF',
            'maxPixels': 1e13
        })
        task.start()


KeyboardInterrupt: 

In [ ]:
# modis_snow_cover = ee.ImageCollection("MODIS/006/MOD10A1").select('NDSI_Snow_Cover')


In [ ]:
# filtered_snow_cover = modis_snow_cover.filterDate('2012-01-01', '2013-12-31')


In [ ]:
# def yearly_composite(year):
#     start_date = ee.Date.fromYMD(year, 1, 1)
#     end_date = start_date.advance(1, 'year')
#     return filtered_snow_cover.filterDate(start_date, end_date).max().clip(roi).set('year', year)

# years = ee.List.sequence(2012, 2013)
# annual_snow_cover = ee.ImageCollection.fromImages(years.map(yearly_composite))


In [11]:
print('where did it all go')

where did it all go
